<a href="https://colab.research.google.com/github/melodyng89/Setia-Alam/blob/master/Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import libraries

In [0]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

#Select target page

In [8]:
url = "https://forum.lowyat.net/topic/3745154/all"
response = requests.get(url)
response

<Response [200]>

#Scrape comments

In [17]:
soup1 = BeautifulSoup(response.text, "html.parser")
result1 = soup1.findAll("div", {'class':'postcolor post_text'})
result1_list = list(result1)

list_result1 = []

for i in range(len(result1_list)):
  list_result1.append(result1[i].get_text())

list_result1

['Panda very curious, if our money is protected by PIDMthen who protected PIDM?Panda cannot brain, PIDM really that rich to protect all Malaysian wealth?   ',
 'up to 250k only ',
 'QUOTE(chokia @ Oct 15 2015, 06:11 PM)up to 250k only250k still a lot of money...  where does the money come from? ',
 'Yes.  Protected.  It is like a form of insurance.  So, they should have underwriters who would cover them. ',
 'Say only, later bankrap, sorry lah. ',
 'QUOTE(Panda @ Oct 15 2015, 06:12 PM)250k still a lot of money...  where does the money come from?donation ',
 "QUOTE(chokia @ Oct 15 2015, 06:13 PM)donationdonation only 26milliondoesn't make sense, thanks for replying anyway... ",
 'QUOTE(Panda @ Oct 15 2015, 06:10 PM)Panda very curious, if our money is protected by PIDMthen who protected PIDM?Panda cannot brain, PIDM really that rich to protect all Malaysian wealth? \xa0 What do you know about PIDM and what money the protect? ',
 'PIDM under Perkasa...anything happen u can refer to Perkas

#Scrape usernames

In [18]:
soup2 = BeautifulSoup(response.text, "html.parser")
result2 = soup2.findAll("span", {'class':'normalname'})
result2_list = list(result2)

list_result2 = []

for i in range(len(result2_list)):
  list_result2.append(result2[i].get_text())

list_result2

['Panda',
 'chokia',
 'Panda',
 'joe_mamak',
 'Matrix',
 'chokia',
 'Panda',
 'grinders',
 'lorrydriverrocks',
 'Panda',
 'grinders',
 'Special Agent',
 'Panda',
 'grinders',
 'patienceGNR',
 'fuzzy',
 'ohman',
 'neoexcaliber',
 'grinders',
 'everest',
 'hihihehe',
 'kenshin9880',
 'patienceGNR',
 'gladfly']

#merge usernams and comments into a table

In [23]:
dataframe = list(zip(list_result2, list_result1))
dataframe = pd.DataFrame(dataframe, columns = ["Person","Comment"])
dataframe.head()

,Person,Comment
0,Panda,"Panda very curious, if our money is protected ..."
1,chokia,up to 250k only
2,Panda,"QUOTE(chokia @ Oct 15 2015, 06:11 PM)up to 250..."
3,joe_mamak,Yes. Protected. It is like a form of insuran...
4,Matrix,"Say only, later bankrap, sorry lah."


#Initialise afinn sentiment analyzer

In [24]:
!pip install afinn
from afinn import Afinn
af = Afinn()

     |████████████████████████████████| 61kB 2.3MB/s 
  Created wheel for afinn: filename=afinn-0.1-cp36-none-any.whl size=53452 sha256=1c8d0e738b9ea4a0fab9eaa39852a7f8f8ac8c6bf9da6168a8faeac82bad0629
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn


#Compute sentiment scores (polarity) and labels

In [0]:
sentiment_scores =[]

for i in range(len(list_result1)):
 sentiment_scores.append(af.score(list_result1[i]))

 sentiment_category = ['positive' if score > 0
                       else 'negative' if score < 0
                       else 'neutral'
                       for score in sentiment_scores]

In [27]:
sentiment_scores

[9.0,
 0.0,
 0.0,
 4.0,
 -1.0,
 2.0,
 2.0,
 10.0,
 0.0,
 1.0,
 0.0,
 0.0,
 2.0,
 0.0,
 3.0,
 2.0,
 0.0,
 -3.0,
 10.0,
 0.0,
 -1.0,
 4.0,
 10.0,
 -10.0]

#Sentiment statistics

In [38]:
df = pd.DataFrame([list(dataframe['Comment']), sentiment_scores, sentiment_category]).T
df.columns = ['Comment', 'sentiment_score', 'sentiment_category'] 
df['sentiment_score'] = df.sentiment_score.astype('float')
df_final = pd.concat([dataframe['Person'],df], axis = 1)
df_final

,Person,Comment,sentiment_score,sentiment_category
0,Panda,"Panda very curious, if our money is protected ...",9.0,positive
1,chokia,up to 250k only,0.0,neutral
2,Panda,"QUOTE(chokia @ Oct 15 2015, 06:11 PM)up to 250...",0.0,neutral
3,joe_mamak,Yes. Protected. It is like a form of insuran...,4.0,positive
4,Matrix,"Say only, later bankrap, sorry lah.",-1.0,negative
5,chokia,"QUOTE(Panda @ Oct 15 2015, 06:12 PM)250k still...",2.0,positive
6,Panda,"QUOTE(chokia @ Oct 15 2015, 06:13 PM)donationd...",2.0,positive
7,grinders,"QUOTE(Panda @ Oct 15 2015, 06:10 PM)Panda very...",10.0,positive
8,lorrydriverrocks,PIDM under Perkasa...anything happen u can ref...,0.0,neutral
9,Panda,"QUOTE(grinders @ Oct 15 2015, 06:20 PM)What do...",1.0,positive


#Compute counts of sentiment category

In [36]:
df_final['sentiment_category']. value_counts()


positive    12
neutral      8
negative     4
Name: sentiment_category, dtype: int64

There are more positive comments than negative comments in this thread


#Summary statistics of sentiment scores

In [40]:
df_final['sentiment_score'].describe()

count    24.000000
mean      1.833333
std       4.517278
min     -10.000000
25%       0.000000
50%       0.500000
75%       3.250000
max      10.000000
Name: sentiment_score, dtype: float64

The average and median sentiment scores are 1.833 and 0.500 respectively.

#Download file

In [0]:
from google.colab import files
df_final.to_csv("comments_lowyat.csv")
files.download("comments_lowyat.csv")